<a href="https://colab.research.google.com/github/leonarddocchagas/data_science/blob/master/Formula_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img width='50%' aling='center' src='https://github.com/leonarddocchagas/data_science/blob/master/projetos/formula1/f1-logo.png?raw=true' />

**Formula 1 World Championship**

Fórmula 1 é a mais popular modalidade de automobilismo do mundo. É a categoria mais avançada do esporte a motor e é regulamentada pela Federação Internacional de Automobilismo.

A história da Fórmula 1 tem início com as competições de Grandes Prêmios disputadas na Europa, no início do século XX, apenas com uma pausa de 1939 até 9 de Setembro de 1945, no Circuito de Silverstone, prosseguindo até a atualidade, sem interrupções e com 72 pistas usadas

*fonte:* https://pt.wikipedia.org/wiki/F%C3%B3rmula_1 

## Sobre os dados

O dados estão separados em diversos arquivos. A depender da análise, será necessário fazer a junção das bases para obter mais informação.

* Dados coletados do keggle
  * Datasets: https://bit.ly/3hDUNLZ

* Lista datasets:
  * circuits.csv
  * constructor_results.csv
  * constructor_standings.csv
  * constructors.csv
  * driver_standings.csv
  * drivers.csv
  * lap_times.csv
  * pit_stops.csv
  * qualifying.csv
  * races.csv
  * results.csv
  * seasons.csv
  * status.csv




##Obtenção e transformação dos dados

**Carrega e verifica dataset**

In [ ]:
# bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# dataset drives
df_driver = pd.read_csv('https://raw.githubusercontent.com/leonarddocchagas/data_science/master/projetos/formula1/drivers.csv')
df_driver.drop(columns=['url', 'number', 'dob', 'driverRef'], inplace=True)
df_driver.loc[:0]

,driverId,code,forename,surname,nationality
0,1,HAM,Lewis,Hamilton,British


In [ ]:
# dataset driver_standings
df_driver_standings = pd.read_csv('https://raw.githubusercontent.com/leonarddocchagas/data_science/master/projetos/formula1/driver_standings.csv')
df_driver_standings.drop(columns=['driverStandingsId'], inplace=True)
df_driver_standings.loc[:0]

,raceId,driverId,points,position,positionText,wins
0,18,1,10.0,1,1,1


In [ ]:
# dataset racer
df_racer = pd.read_csv('https://raw.githubusercontent.com/leonarddocchagas/data_science/master/projetos/formula1/races.csv')
df_racer.drop(columns=['url', 'time'], inplace=True)
df_racer.head(1)

,raceId,year,round,circuitId,name,date
0,1,2009,1,1,Australian Grand Prix,2009-03-29


Com os dataset separados, vamos unir o `df_driver`, `df_driver_standings`, `df_race`, assim, temos uma visão geral a performance de pontuação dos pilotos. 

In [ ]:
# juntando os datasets com (left join)
df_full = pd.merge(df_driver_standings, df_driver, how='left', on='driverId')
df_full = pd.merge(df_full, df_racer, how='left', on='raceId')

# deletando colunas
df_full.drop(columns=['raceId', 'driverId', 'circuitId'], inplace=True)
df_full.head()

,points,position,positionText,wins,code,forename,surname,nationality,year,round,name,date
0,10.0,1,1,1,HAM,Lewis,Hamilton,British,2008,1,Australian Grand Prix,2008-03-16
1,8.0,2,2,0,HEI,Nick,Heidfeld,German,2008,1,Australian Grand Prix,2008-03-16
2,6.0,3,3,0,ROS,Nico,Rosberg,German,2008,1,Australian Grand Prix,2008-03-16
3,5.0,4,4,0,ALO,Fernando,Alonso,Spanish,2008,1,Australian Grand Prix,2008-03-16
4,4.0,5,5,0,KOV,Heikki,Kovalainen,Finnish,2008,1,Australian Grand Prix,2008-03-16
